## Summary

**Framework**:

Question -> Function_call(LLM1) -> input -> IAMs climate models -> output -> LLM2 -> Answer

**Paper Draft**:

https://www.overleaf.com/1225397724nhhpgfjszczg#a93b66

**Methods**

- Level 1: Simple QA
- Level 2: IPCC related QA or claim fact-checking https://huggingface.co/datasets/rlacombe/ClimateX
- Level 3: General climate QA

## Level 1

### Level 1 simple Questions

- Based on climate model result to create a question dataset

In [1]:
import numpy as np
import pandas as pd

In [393]:
# output_variables = ["Surface Air Temperature Change",
#         "Atmospheric Concentrations|CO2",
#         "Effective Radiative Forcing",
#         "Effective Radiative Forcing|CO2",
#         "Effective Radiative Forcing|Aerosols",
#         "Effective Radiative Forcing|Aerosols|Direct Effect|BC",
#         "Effective Radiative Forcing|Aerosols|Direct Effect|OC",
#         "Effective Radiative Forcing|Aerosols|Direct Effect|SOx",
#         "Effective Radiative Forcing|Aerosols|Direct Effect",
#         "Effective Radiative Forcing|Aerosols|Indirect Effect"]

# unit = ["K", "ppm", "W / m^2", "W / m^2", "W / m^2", "W / m^2", "W / m^2", "W / m^2", "W / m^2", "W / m^2"]

# unit = {"Surface Air Temperature Change": "K",
#         "Atmospheric Concentrations|CO2": "ppm",
#         "Effective Radiative Forcing": "W / m^2",
#         "Effective Radiative Forcing|CO2": "W / m^2",
#         "Effective Radiative Forcing|Aerosols": "W / m^2",
#         "Effective Radiative Forcing|Aerosols|Direct Effect|BC": "W / m^2",
#         "Effective Radiative Forcing|Aerosols|Direct Effect|OC": "W / m^2",
#         "Effective Radiative Forcing|Aerosols|Direct Effect|SOx": "W / m^2",
#         "Effective Radiative Forcing|Aerosols|Direct Effect": "W / m^2",
#         "Effective Radiative Forcing|Aerosols|Indirect Effect": "W / m^2",
#         }

year = list(range(2024, 2101))

# print(year)

# scenario = ["ssp119", "ssp126", "ssp245", "ssp370", "ssp370-lowNTCF-aerchemmip", "ssp370-lowNTCF-gidden", "ssp434", "ssp460", "ssp534-over", "ssp585"]

scenario = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp460', 'ssp585']

region = ["World"]

### function_call
- setup chatgpt api
- Translate the climate model into openai function

In [2]:
# OpenAI API
from getpass import getpass
import os
import openai
from openai import OpenAI

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
new_api_key = getpass()

os.environ["OPENAI_API_KEY"] = new_api_key
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

 ········


In [583]:
import json

# question = "What is the World Surface Air Temperature Change under ssp245 in 2033?"
# question = "What is the World Surface Air Temperature Change under ssp585 in 23 years?"

# question = "When the world surface air temperature will change +2 degree?"

# question = "What is the World Sea level Change in 50 years?"

question = "When the world mean sea level will increase 500mm?"

# Step 1: send the conversation and available functions to the model
messages = [{"role": "user", "content": question}]

In [584]:
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_temperature_change_value",
                "description": "Get the Surface Air Temperature Change data in given year and scenario.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "variable": {
                            "type": "string",
                            "description": "a range of climate-related metrics",
                            "enum": ["Surface Air Temperature Change",
                            "Atmospheric Concentrations|CO2",
                            "Effective Radiative Forcing",
                            "Effective Radiative Forcing|CO2",
                            "Effective Radiative Forcing|Aerosols",
                            "Effective Radiative Forcing|Aerosols|Direct Effect|BC",
                            "Effective Radiative Forcing|Aerosols|Direct Effect|OC",
                            "Effective Radiative Forcing|Aerosols|Direct Effect|SOx",
                            "Effective Radiative Forcing|Aerosols|Direct Effect",
                            "Effective Radiative Forcing|Aerosols|Indirect Effect",
                            ]
                        },
                        "year": {"type": "string", "description": "years starting from 2024 to 2100"},
                        "scenario": {
                            "type": "string",
                            "description": "a list of climate scenario settings",
                            "enum": ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp460', 'ssp585']
                        },
                    },
                    "required": ["year", "scenario"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_temperature_change_year",
                "description": "Get the time when Surface Air Temperature Change arrive a degree.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "variable": {
                            "type": "string",
                            "description": "a range of climate-related metrics",
                            "enum": ["Surface Air Temperature Change"]
                        },
                        "temperature": {"type": "string", "description": "target temperature change"},
                        "scenario": {
                            "type": "string",
                            "description": "a list of climate scenario settings, by default ssp245",
                            "enum": ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp460', 'ssp585']
                        },
                    },
                    "required": ["temperature"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_sea_level",
                "description": "Get the sea level change data given year",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "year": {"type": "string", "description": "years starting from 2024 to 2100"},
                    },
                    "required": ["year"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_sea_level_year",
                "description": "Get the time when Sea level Change arrive a height.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "value": {"type": "string", "description": "sea level change value the unit is mm"},
                        "scenario": {
                            "type": "string",
                            "description": "a list of climate scenario settings, by default no_restriction",
                            "enum": ['no_restriction', 'COP 26', 'ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp460', 'ssp585']
                        },
                    },
                    "required": ["value"],
                },
            },
        }
    ]

In [585]:
# df = pd.read_csv('/Users/ylpan/IPCC_Report_Models/magicc7_res_World_CI_2024_2100.csv')
# # df['variable'] == 'Sea Level Change'
# # df['scenario'] == 'ssp370'
# col = ['model','quantile', 'variable','unit', 'scenario', '2045']
# df[col][df['variable'] == 'Sea Level Change'][df['scenario'] == 'ssp370']

In [586]:
# Example climate function to return the data

def get_temperature_change_value(year, scenario, variable = "Surface Air Temperature Change", temperature = 0):
    """Get the Surface Air Temperature Change data in given year and scenario."""
    df = pd.read_csv('/Users/ylpan/IPCC_Report_Models/magicc7_res_World_CI_2024_2100.csv')
    col = ['model','quantile', 'variable','unit', 'scenario', year]
    df[col]
    # value = magicc_res.filter(
    # variable=variables,
    # year=year,
    # scenario='ssp126',
    # region='World').values[0][0]
    
    return df[col][(df['variable'] == variable) & (df['scenario'] == scenario)].to_json()

def get_temperature_change_year(temperature, scenario = ['ssp245'], variable = "Surface Air Temperature Change", year = 0):
    """Get the time when Surface Air Temperature Change arrive a degree."""
    df = pd.read_csv('/Users/ylpan/IPCC_Report_Models/magicc7_res_World_CI_2024_2100.csv', index_col=None)
    years = [str(year) for year in range(2024, 2101)]
    col = ['model','quantile', 'variable','unit', 'scenario'] + years
    df_T = df[col][(df['quantile'] == 0.5)&(df['variable'] == 'Surface Air Temperature Change')][['scenario']+years].T
    df_T.columns = df_T.iloc[0]
    df_T = df_T[1:]
    # df.columns = df.iloc[0]
    # col = ['ssp126', 'ssp585', scenario]
    df_T = df_T[scenario]
    lower_bound = float(temperature) - 0.2
    upper_bound = float(temperature) + 0.2
    
    return df_T[(df_T >= lower_bound) & (df_T <= upper_bound)].to_json()


def get_sea_level(year, variable = 'Sea Level Change'):
    """Get the sea level change data in given year."""
    df = pd.read_csv('/Users/ylpan/IPCC_Report_Models/magicc7_res_World_CI_2024_2100.csv')
    col = ['model','quantile', 'variable','unit', 'scenario', year]
    df[col]
    # value = magicc_res.filter(
    # variable=variables,
    # year=year,
    # scenario='ssp126',
    # region='World').values[0][0]
    
    return df[col][(df['variable'] == variable)].to_json()

def get_sea_level_year(value, variable = 'Sea Level Change', scenario = 'no_restriction'):
    """Get the time when Sea level Change arrive a height."""
    df = pd.read_csv('/Users/ylpan/IPCC_Report_Models/magicc7_res_World_CI_2024_2100.csv', index_col=None)
    years = [str(year) for year in range(2024, 2101)]
    col = ['model','quantile', 'variable','unit', 'scenario'] + years
    df_T = df[col][(df['quantile'] == 0.5)&(df['variable'] == 'Sea Level Change')][['scenario']+years].T
    df_T.columns = df_T.iloc[0]
    df_T = df_T[1:]
    # df_T = df_T[scenario]
    lower_bound = float(value) - 75
    upper_bound = float(value) + 75
    
    return df_T[(df_T >= lower_bound) & (df_T <= upper_bound)].to_json()
print(get_sea_level_year('600'))

{"no_restriction ":{"2024":null,"2025":null,"2026":null,"2027":null,"2028":null,"2029":null,"2030":null,"2031":null,"2032":null,"2033":null,"2034":null,"2035":null,"2036":null,"2037":null,"2038":null,"2039":null,"2040":null,"2041":null,"2042":null,"2043":null,"2044":null,"2045":null,"2046":null,"2047":null,"2048":null,"2049":null,"2050":null,"2051":null,"2052":null,"2053":null,"2054":null,"2055":null,"2056":null,"2057":null,"2058":null,"2059":null,"2060":null,"2061":null,"2062":null,"2063":null,"2064":null,"2065":null,"2066":null,"2067":null,"2068":null,"2069":null,"2070":null,"2071":null,"2072":null,"2073":null,"2074":null,"2075":null,"2076":null,"2077":null,"2078":null,"2079":null,"2080":null,"2081":null,"2082":null,"2083":null,"2084":null,"2085":525.3923453,"2086":535.340276,"2087":545.399216,"2088":555.5700706,"2089":565.8537532,"2090":576.251186,"2091":586.7632993,"2092":597.3910323,"2093":608.1353326,"2094":618.9971566,"2095":629.9774692,"2096":641.0772445,"2097":652.2974651,"209

In [587]:
# df = pd.read_csv('/Users/ylpan/IPCC_Report_Models/magicc7_res_World_CI_2024_2100(2).csv', index_col=None)
# years = [str(year) for year in range(2024, 2101)]
# col = ['model','quantile', 'variable','unit', 'scenario'] + years
# df_T = df[col][(df['quantile'] == 0.5)&(df['variable'] == 'Sea Level Change')][['scenario']+years].T
# df_T.columns = df_T.iloc[0]
# df_T = df_T[1:]

# # df.columns = df.iloc[0]
# # print(df_T)

# df_T['ssp119']
# # lower_bound = 1.9
# # upper_bound = 2.1

# # df_T[(df_T >= lower_bound) & (df_T <= upper_bound)]



2024     95.199996
2025    100.556903
2026    105.736181
2027    110.746531
2028    115.625262
           ...    
2096    454.435687
2097    460.302864
2098    466.199846
2099    472.126847
2100    478.084127
Name: ssp119, Length: 77, dtype: object

In [588]:
response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )

In [589]:
response_message = response.choices[0].message

In [590]:
classification_result = eval(response_message.tool_calls[0].function.arguments)
# print(classification_result['variable'])
# print(classification_result['year'])
# print(classification_result['scenario'])
print(classification_result)

{'value': '500'}


In [591]:
tool_calls = response_message.tool_calls

In [592]:
print(tool_calls)

[ChatCompletionMessageToolCall(id='call_Qnxj3TceHBSnQmVkBfZjvK5X', function=Function(arguments='{"value":"500"}', name='get_sea_level_year'), type='function')]


In [597]:

# Step 2: check if the model wanted to call a function

if tool_calls:
        messages = [{"role": "user", "content": question}]
        messages = [
        {"role": "system", "content": 'You are a helpful climate assistant. You will answer the question about temperature change and sea level change. The temperature change is based on 1765 year. The sea level change is based on 1996.'},
        {"role": "user", "content": question}
        ]
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_temperature_change_value": get_temperature_change_value,
            "get_temperature_change_year": get_temperature_change_year,
            "get_sea_level": get_sea_level,
            "get_sea_level_year": get_sea_level_year
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        print(messages)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                # variable=function_args.get("variable"),
                # year=function_args.get("year"),
                # scenario = function_args.get("scenario"),
                # temperature = function_args.get("temperature")
                # value = function_args.get("value")
            )
            
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        print(messages)
        
        second_response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response

[{'role': 'system', 'content': 'You are a helpful climate assistant. You will answer the question about temperature change and sea level change. The temperature change is based on 1765 year. The sea level change is based on 1996.'}, {'role': 'user', 'content': 'When the world mean sea level will increase 500mm?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Qnxj3TceHBSnQmVkBfZjvK5X', function=Function(arguments='{"value":"500"}', name='get_sea_level_year'), type='function')])]
[{'role': 'system', 'content': 'You are a helpful climate assistant. You will answer the question about temperature change and sea level change. The temperature change is based on 1765 year. The sea level change is based on 1996.'}, {'role': 'user', 'content': 'When the world mean sea level will increase 500mm?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='c

In [598]:
# function_response

In [599]:
print(second_response.choices[0].message.content)

According to different climate scenarios (SSP), the world mean sea level is projected to increase by 500mm at different times:

- **No Restriction**: Around 2082
- **COP26**: Around 2090
- **SSP1-1.9**: This scenario does not reach a 500mm increase by 2100.
- **SSP1-2.6**: Late 2090s (approximately 2097-2098)
- **SSP2-4.5**: Around 2079-2080
- **SSP3-7.0**: Late 2070s (approximately 2077-2078)
- **SSP4-6.0**: Early 2070s (approximately 2073-2074)
- **SSP5-8.5**: Early 2070s (approximately 2070-2072) 

These projections depend greatly on global greenhouse gas emissions and climate policy actions taken in the coming years.


In [596]:
from IPython.display import Markdown, display

display(Markdown(second_response.choices[0].message.content))

Based on the calculations provided for various scenarios, the year by which the world mean sea level will increase by 500mm varies depending on the climate change scenario:

1. **No Restriction Scenario**: The sea level will increase by 500mm around the year **2083**.
2. **COP26**: The sea level will increase by 500mm around the year **2093**.
3. **SSP1-1.9** (Stringent Emissions Mitigation): The sea level is projected to be around 478 mm by **2100**, and might not reach the 500mm point within this century.
4. **SSP1-2.6**: The sea level will increase by 500mm around the year **2093**.
5. **SSP2-4.5** (Intermediate Scenario): The sea level is projected to increase by 500mm around the year **2085**.
6. **SSP3-7.0**: The sea level is projected to increase by 500mm around the year **2076**.
7. **SSP4-6.0**: The sea level is projected to increase by 500mm around the year **2080**.
8. **SSP5-8.5**: The sea level is projected to increase by 500mm around the year **2074**.

These projections are based on different assumptions about future greenhouse gas emissions and socio-economic development pathways. The "No Restriction" case usually represents scenarios with unchecked emissions, whereas the SSP (Shared Socioeconomic Pathways) scenarios cover a range of possible futures based on varying levels of climate action and mitigation efforts.

## Level 3 

In [600]:
Q1 = 'What is the best estimate of the equilibrium climate sensitivity?'
Q2 = 'Is it possible the Arctic will become ice free at some points before 2050?'
Q3 = 'How likely will the world see runaway ice loss from the Antarctic ice sheets by 2100?'
Q4 = 'To what extent has climate change impacted ice sheets in both polar regions?'
Q5 = 'Will coastal New York see the impact of Greenland ice sheet melting by 2100?'
Q6 = 'What will be the likely changes to weather and climate extremes in coastal New York around 2050 if the world take no action to reduce greenhouse gas emissions?'
Q7 = 'What are the primary evidence and how robust is it that human activities caused the observed global warming since the pre-industrial period?'


In [630]:
def get_chatiams(question):
    messages = [{"role": "user", "content": question}]
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
# Step 2: check if the model wanted to call a function

    if tool_calls:
        messages = [{"role": "user", "content": question}]
        # messages = [
        # {"role": "system", "content": 'You are a helpful climate assistant. You will answer the question about temperature change and sea level change. The temperature change is based on 1765 year. The sea level change is based on 1996.'},
        # {"role": "user", "content": question}
        # ]
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_temperature_change_value": get_temperature_change_value,
            "get_temperature_change_year": get_temperature_change_year,
            "get_sea_level": get_sea_level,
            "get_sea_level_year": get_sea_level_year
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        print(messages)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                # variable=function_args.get("variable"),
                # year=function_args.get("year"),
                # scenario = function_args.get("scenario"),
                # temperature = function_args.get("temperature"),
                # value = function_args.get("value")
            )
            
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        print(messages)
        
        second_response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response.choices[0].message.content
    return response.choices[0].message.content

In [619]:
display(Markdown(get_chatiams(Q1)))

The best estimate of the equilibrium climate sensitivity (ECS) is around 3°C. Equilibrium climate sensitivity is defined as the global average surface warming following a doubling of carbon dioxide (CO₂) concentrations relative to pre-industrial levels. It is a key measure of the Earth's climate's responsiveness to increased greenhouse gas concentrations.

While the range of ECS is generally considered to be between 1.5°C to 4.5°C, the best estimate of 3°C is based on a combination of climate models, empirical data, and paleoclimate evidence.

In [620]:
display(Markdown(get_chatiams(Q2)))

Yes, it is possible that the Arctic could become ice-free during some parts of the year before 2050. Climate models and observational data suggest that the Arctic is warming at a rate twice as fast as the global average, leading to a rapid decline in sea ice. Summer sea ice has been particularly affected, and some scenarios project the possibility of ice-free conditions during the summer months as early as the 2030s or 2040s, depending on the level of greenhouse gas emissions and other factors.

In [621]:
display(Markdown(get_chatiams(Q3)))

The likelihood of runaway ice loss from the Antarctic ice sheets by 2100 depends on various factors including greenhouse gas emissions, global temperature increase, and their subsequent impacts on atmospheric and oceanic conditions. Higher levels of warming increase the risk of significant ice loss.

One key metric to consider is global sea level rise, which is influenced by melting ice sheets. By examining projected sea level changes under different climate scenarios, we can get insights into potential risks of runaway ice loss from the Antarctic ice sheets.

Would you like me to retrieve the projected sea level changes by 2100 under multiple climate scenarios to better understand this risk?

In [622]:
display(Markdown(get_chatiams(Q4)))

Climate change has had significant impacts on ice sheets in both the Antarctic and Arctic regions, contributing to a rise in global sea levels and affecting global climate patterns. Here are some specific impacts:

### Antarctic Ice Sheet
1. **Melting and Ice Loss**: The Antarctic ice sheet is losing ice mass at an accelerating rate, primarily due to warm ocean currents melting the ice from below. The western Antarctic ice sheet is particularly vulnerable.
2. **Sea Level Rise**: The ice loss from Antarctica contributes directly to global sea level rise. Studies estimate that Antarctic ice melt has contributed to a significant portion of the observed sea level rise over the past few decades.
3. **Changing Ice Dynamics**: The ice flow dynamics are changing, with some regions seeing faster glacier flow to the sea, increasing the rate of ice discharge into the ocean.
4. **Ice Shelf Disintegration**: Several ice shelves have disintegrated or thinned dramatically, which can destabilize the grounded ice behind them leading to increased ice flow into the sea.

### Arctic Ice Sheet (Greenland Ice Sheet)
1. **Melting and Ice Loss**: The Greenland ice sheet is also losing ice mass at an accelerating rate due to higher air temperatures and changes in precipitation patterns.
2. **Glacial Retreat**: Many of Greenland’s glaciers have retreated significantly. Surface melting has increased and ice is being lost through calving of icebergs into the ocean.
3. **Sea Level Rise**: Greenland's ice melt is a major contributor to global sea level rise. It has been estimated that Greenland ice loss is contributing to increasing sea levels at an accelerating pace.
4. **Warming Amplification**: The Arctic region is warming at approximately twice the rate of the global average, a phenomenon known as Arctic amplification. This accelerates the melting process.
5. **Permafrost Thawing**: Although not part of the ice sheet, thawing permafrost releases greenhouse gases such as methane, which further contribute to global warming.

### Observational Data
- Satellite observations have been instrumental in monitoring changes in ice mass and extent. Both NASA's GRACE (Gravity Recovery and Climate Experiment) mission and ESA's CryoSat-2 mission have provided valuable data.
- Measurements indicate that the ice loss in both the Antarctic and Greenland ice sheets has been accelerating over recent decades.

### Future Projections
- If current trends continue, both ice sheets will continue to lose mass, contributing even more to sea level rise. 
- Different climate scenarios project varying degrees of impact, but persistent warming will likely result in continued ice sheet destabilization and higher rates of sea level rise.

These impacts can have serious implications for coastal communities, ecosystems, and global weather patterns. Efforts to curb greenhouse gas emissions are crucial to mitigating these effects.

In [623]:
display(Markdown(get_chatiams(Q5)))

Yes, coastal New York is expected to experience the impacts of the melting Greenland ice sheet by 2100. The melting of the Greenland ice sheet contributes significantly to global sea level rise, which in turn affects coastal areas around the world, including New York.

To provide a more detailed and specific answer regarding the extent of sea level rise by 2100, I can retrieve sea level change data for that year. Would you like me to do that?

In [626]:
display(Markdown(get_chatiams(Q6)))

[{'role': 'user', 'content': 'What will be the likely changes to weather and climate extremes in coastal New York around 2050 if the world take no action to reduce greenhouse gas emissions?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_If65muAJb31ZgckRdslWDM59', function=Function(arguments='{"variable":"Surface Air Temperature Change","year":"2050","scenario":"ssp585"}', name='get_temperature_change_value'), type='function')])]
[{'role': 'user', 'content': 'What will be the likely changes to weather and climate extremes in coastal New York around 2050 if the world take no action to reduce greenhouse gas emissions?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_If65muAJb31ZgckRdslWDM59', function=Function(arguments='{"variable":"Surface Air Temperature Change","year":"2050","scenario":"ssp585"}', name='get_temperature_change_v

If no action is taken to reduce greenhouse gas emissions, the climate model projections under the high-emission scenario SSP5-8.5 suggest significant increases in surface air temperatures for coastal New York by around 2050. The median estimate indicates an increase of approximately 2.39°C above the baseline temperature levels.

Here are some likely changes to weather and climate extremes in coastal New York around 2050 under these conditions:

1. **Increased Temperature Extremes**:
   - More frequent and intense heatwaves, with higher maximum temperatures.
   - Warmer night-time temperatures, reducing the occurrence of cool nights.

2. **Sea-Level Rise**:
   - Rising sea levels will contribute to more frequent and severe coastal flooding, especially during high tides and storm surges.
   - Increased risk of coastal erosion and loss of land due to higher sea levels coupled with storm events.

3. **More Intense Storms**:
   - Potential for more intense tropical cyclones (hurricanes) with higher wind speeds and more heavy rainfall, although the frequency of these storms can vary.
   - Greater likelihood of damaging storm surges due to a combination of sea-level rise and intense storm activity.

4. **Increased Precipitation Extremes**:
   - More frequent and severe heavy rainfall events, leading to a higher risk of flash flooding and urban flooding.
   - Possible changes in seasonal precipitation patterns, potentially causing prolonged periods of both drought and intense rainfall.

5. **Changes in Ocean Conditions**:
   - Ocean warming and acidification can impact coastal ecosystems, fisheries, and biodiversity.
   - More frequent and widespread harmful algal blooms, affecting water quality and marine life.

Adapting to these changes will require significant efforts in updating infrastructure, improving emergency response systems, and planning for resilient coastal development.

In [627]:
display(Markdown(get_chatiams(Q7)))

The primary evidence that human activities have caused the observed global warming since the pre-industrial period comes from a convergence of multiple lines of robust, consistent scientific evidence. The following points outline this evidence:

1. **Temperature Records**:
    - Observational data from a variety of sources (land, sea, and atmospheric) shows an unequivocal increase in global mean surface temperatures since the late 19th century.
    - Instrumental records (thermometers, satellites) indicate that the last few decades have been the warmest in the past 1400 years.

2. **Greenhouse Gas Concentrations**:
    - Ice core samples reveal historical atmospheric concentrations of greenhouse gases (GHGs) such as CO2, CH4, and N2O. GHG concentrations have increased dramatically since the Industrial Revolution due to human activities like burning fossil fuels, deforestation, and agriculture.
    - Modern measurements confirm that current levels of CO2 are unprecedented in at least the last 800,000 years.

3. **Radiative Forcing**:
    - The concept of radiative forcing helps quantify the influence of factors (like GHGs) that affect the balance of incoming and outgoing energy in the Earth's atmosphere. Positive radiative forcing from increased GHGs results in warming.
    - Human activities account for a significant portion of the positive radiative forcing observed, much more than natural factors such as solar variability or volcanic activity.

4. **Climate Models**:
    - Coupled climate models that include both natural and anthropogenic factors can replicate historical temperature trends accurately.
    - Models that exclude human influences fail to replicate the observed warming, indicating the crucial role of human activities in recent climate changes.

5. **Paleoclimate Evidence**:
    - Studies of past climate conditions (paleoclimatology) using proxy data (such as tree rings, ice cores, and sediment layers) show that recent warming is unusual in the context of the last two millennia.

6. **Patterns of Warming**:
    - The observed geographical and vertical patterns of atmospheric warming match those predicted by models that include human influences. For example, the troposphere (lower atmosphere) is warming while the stratosphere (upper atmosphere) is cooling, which is consistent with GHG-induced warming and not with other potential causes.

7. **Ocean Heat Content**:
    - The world's oceans have absorbed over 90% of the excess heat generated by GHGs, leading to measurable increases in ocean heat content. This is consistent with human-induced global warming and is less likely to be explained by natural variability.

8. **Sea Level Rise**:
    - Global sea levels are rising due to the thermal expansion of seawater as it warms and the melting of ice sheets and glaciers. The rate of sea level rise has accelerated in recent decades, correlating with increased global temperatures.

9. **Attribution Studies**:
    - Formal attribution studies use statistical methods to separate human influences from natural influences on climate. These studies consistently find that the majority of the observed warming since the mid-20th century is due to human activities.

### Robustness of Evidence
The robustness of the evidence is extremely high:

- **Consilience of Evidence**: The agreement of evidence from multiple independent lines of inquiry makes the conclusion that human activities are the dominant cause of recent global warming very robust.
- **Peer-Reviewed Research**: Hundreds of peer-reviewed studies support the conclusion, and it is endorsed by major scientific organizations worldwide, including the Intergovernmental Panel on Climate Change (IPCC), National Aeronautics and Space Administration (NASA), and the National Oceanic and Atmospheric Administration (NOAA).
- **Consensus**: There is an overwhelming consensus among climate scientists about the reality and anthropogenic causes of global warming. More than 97% of actively publishing climate scientists agree that climate-warming trends over the past century are extremely likely due to human activities.

In summary, the primary evidence points strongly towards human activities being the main driver of the observed global warming since the pre-industrial period, and the robustness of this evidence is extremely high due to its consistency, broad scientific consensus, and extensive peer-reviewed research.

## Basic Chat UI

In [3]:
import panel as pn
pn.extension()

In [4]:
def get_chatiams(question, user, chat_interface):
    messages = [{"role": "user", "content": question}]
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
# Step 2: check if the model wanted to call a function

    if tool_calls:
        messages = [{"role": "user", "content": question}]
        # messages = [
        # {"role": "system", "content": 'You are a helpful climate assistant. You will answer the question about temperature change and sea level change. The temperature change is based on 1765 year. The sea level change is based on 1996.'},
        # {"role": "user", "content": question}
        # ]
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_temperature_change_value": get_temperature_change_value,
            "get_temperature_change_year": get_temperature_change_year,
            "get_sea_level": get_sea_level,
            "get_sea_level_year": get_sea_level_year
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        print(messages)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                # variable=function_args.get("variable"),
                # year=function_args.get("year"),
                # scenario = function_args.get("scenario"),
                # temperature = function_args.get("temperature"),
                # value = function_args.get("value")
            )
            
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        print(messages)
        
        second_response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response.choices[0].message.content
    return response.choices[0].message.content

In [5]:
chat_interface = pn.chat.ChatInterface(
    callback=get_chatiams, 
    callback_user="ChatIAMs"
)
chat_interface

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'TextInput': TextInput(cs...}, callback=<function get_chatiams a..., callback_user='ChatIAMs', show_button_name=True, sizing_mode='stretch_width', widgets=[TextInput(css_classes=['c...])